In [3]:
######################## Create a Travel Itinerary Map ########################

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps
gmaps.configure(api_key=g_key)

In [2]:
# Read the WeatherPy_vacation.csv into a DataFrame.
vacation_df = pd.read_csv("../Vacation_Search/WeatherPy_vacation.csv")
vacation_df.head()

,City_ID,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,2,Bengkulu,ID,76.86,broken clouds,-3.8004,102.2655,Grage Hotel Bengkulu
1,5,Saint-Philippe,RE,84.20,light rain,-21.3585,55.7679,"Chambres d'hôte ""La Trinité"""
2,7,Avarua,CK,77.00,light rain,-21.2078,-159.7750,Paradise Inn
3,10,Atuona,PF,80.29,broken clouds,-9.8000,-139.0333,Villa Enata
4,11,Rikitea,PF,77.47,overcast clouds,-23.1203,-134.9692,Pension Maro'i


In [3]:
# Rename the column "Current Description" to "Current Weather".
vacation_df.columns = ["City_ID","City","Country","Max Temp","Current Weather","Lat","Lng","Hotel Name"]
vacation_df.head()

,City_ID,City,Country,Max Temp,Current Weather,Lat,Lng,Hotel Name
0,2,Bengkulu,ID,76.86,broken clouds,-3.8004,102.2655,Grage Hotel Bengkulu
1,5,Saint-Philippe,RE,84.20,light rain,-21.3585,55.7679,"Chambres d'hôte ""La Trinité"""
2,7,Avarua,CK,77.00,light rain,-21.2078,-159.7750,Paradise Inn
3,10,Atuona,PF,80.29,broken clouds,-9.8000,-139.0333,Villa Enata
4,11,Rikitea,PF,77.47,overcast clouds,-23.1203,-134.9692,Pension Maro'i


In [4]:
# Using the template add hotel name, the city name, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Weather} and {Max Temp} °F</dd>  
</dl>
"""

# Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in vacation_df.iterrows()]

# Get the latitude and longitude from each row and store in a new DataFrame.
locations = vacation_df[["Lat", "Lng"]]

In [5]:
# Add a marker layer for each city to the map.
max_temp = vacation_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp, dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)


# Display the figure
fig


Figure(layout=FigureLayout(height='420px'))

In [7]:
# Set the city to start and end your trip
vacation_start = vacation_df.loc[vacation_df["City"] == "Bengkulu"]
vacation_end = vacation_df.loc[vacation_df["City"] == "Bengkulu"]
vacation_end

# Set the cities to stop at
vacation_stop1 = vacation_df.loc[vacation_df["City"] == "Kupang"]
vacation_stop2 = vacation_df.loc[vacation_df["City"] == "Gorontalo"] 
vacation_stop3 = vacation_df.loc[vacation_df["City"] == "Tual"]

In [8]:
# Get the lat and lng for the start and end of the trip
start = vacation_start["Lat"].to_numpy()[0], vacation_start["Lng"].to_numpy()[0]
end = vacation_start["Lat"].to_numpy()[0], vacation_start["Lng"].to_numpy()[0]
print(f'Start: {start}, End: {end}')


# Get the lat and lng for the stops
stop1 = vacation_stop1["Lat"].to_numpy()[0], vacation_stop1["Lng"].to_numpy()[0] 
stop2 = vacation_stop2["Lat"].to_numpy()[0], vacation_stop2["Lng"].to_numpy()[0]
stop3 = vacation_stop3["Lat"].to_numpy()[0], vacation_stop3["Lng"].to_numpy()[0]

print(f'Stop 1: {stop1}, Stop 2: {stop2}, Stop 3: {stop3}')

Start: (-3.8004, 102.2655), End: (-3.8004, 102.2655)
Stop 1: (-10.1667, 123.5833), Stop 2: (0.5412, 123.0595), Stop 3: (-5.6667, 132.75)


In [9]:
# Create Map with route
fig = gmaps.figure()
city_itinerary = gmaps.directions_layer(
        start, end, waypoints=[stop1, stop2, stop3],
        travel_mode='DRIVING')

fig.add_layer(city_itinerary)
fig


Figure(layout=FigureLayout(height='420px'))

In [10]:
# To create a marker layer map between the four cities.
#  Combine the four city DataFrames into one DataFrame using the concat() function.
itinerary_df = pd.concat([vacation_start,vacation_stop1,vacation_stop2,vacation_stop3],ignore_index=True)
itinerary_df

,City_ID,City,Country,Max Temp,Current Weather,Lat,Lng,Hotel Name
0,2,Bengkulu,ID,76.86,broken clouds,-3.8004,102.2655,Grage Hotel Bengkulu
1,146,Kupang,ID,79.63,overcast clouds,-10.1667,123.5833,Hotel On The Rock by Prasanthi
2,103,Gorontalo,ID,75.09,overcast clouds,0.5412,123.0595,Maqna Hotel by Prasanthi
3,73,Tual,ID,83.91,overcast clouds,-5.6667,132.7500,Grand Vilia Hotel


In [11]:
# Using the template add hotel name, the city name, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Weather} and {Max Temp} °F</dd>  
</dl>
"""

# Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in itinerary_df.iterrows()]

# Get the latitude and longitude from each row and store in a new DataFrame.
locations = itinerary_df[["Lat", "Lng"]]

In [12]:
# Add a marker layer for each city to the map.
max_temp = itinerary_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp, dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# Display the figure
fig 

Figure(layout=FigureLayout(height='420px'))